In [1]:
!pip install -q llama-cpp-python huggingface-hub gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00


In [2]:
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf"
)
print("✅ Model downloaded to:", model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

✅ Model downloaded to: /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf


In [3]:
from llama_cpp import Llama

llm = Llama(
    model_path=model_path,
    n_ctx=4096,  # You can increase this if needed
    n_threads=8,
    verbose=False
)


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [6]:
def chat_with_mistral(prompt):
    formatted_prompt = f"[INST] {prompt.strip()} [/INST]"
    response = llm.create_completion(
        prompt=formatted_prompt,
        max_tokens=256,
        temperature=0.7
    )
    return response["choices"][0]["text"].strip()


In [7]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Chat with Mistral 7B Instruct (GGUF Model in Colab)")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message", placeholder="Type your question...", lines=1)

    def respond(user_message, chat_history):
        bot_reply = chat_with_mistral(user_message)
        chat_history.append((user_message, bot_reply))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    gr.Button("Clear").click(lambda: [], None, chatbot, queue=False)

# Step 6: Launch Gradio with a public URL
demo.launch(share=True)

<ipython-input-7-1232419934>:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3dc2fc3ae76280b9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
